In [1]:
import json
import pandas as pd
import re
import math
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, LabelBinarizer
import numpy as np

In [2]:
with open('data/default-cards-20230612090651.json') as f:
    cards_json = json.load(f)

In [3]:
len(cards_json)

83971

In [4]:
double_faced_cards = []

for card in cards_json:
    if '//' in card['name']:
        double_faced_cards.append(card)

In [5]:
len(double_faced_cards)

2895

## Preparação do conjunto de dados

### Carga e tratamento inicial do conjunto de dados de cartas

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
raw_cards = pd.read_json('data/default-cards-20230612090651.json')
print(f'Starter raw card set length ---> : {len(raw_cards.index)}')

Starter raw card set length ---> : 83971


In [8]:
card_set_drop_reprints = raw_cards.query('reprint != True')
print(f'Number of removed cards ---> : {len(raw_cards.index) - len(card_set_drop_reprints.index)}')
print(f'Current card set length ---> : {len(card_set_drop_reprints.index)}')

Number of removed cards ---> : 50057
Current card set length ---> : 33914


In [9]:
card_set_drop_digital_content = card_set_drop_reprints.query('digital != True')
print(f'Number of removed cards ---> : {len(card_set_drop_reprints.index) - len(card_set_drop_digital_content.index)}')
print(f'Current card set length ---> : {len(card_set_drop_digital_content.index)}')

Number of removed cards ---> : 746
Current card set length ---> : 33168


In [10]:
card_set_drop_tokens = card_set_drop_digital_content.query('not type_line.str.contains("Token")')
card_set_drop_tokens = card_set_drop_tokens.query('not type_line.str.contains("Tolkien")')
print(f'Current card set length without tokens ---> : {len(card_set_drop_tokens.index)}')

Current card set length without tokens ---> : 32534


#### Remoção de cartas de sets inválidos

In [11]:
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] != 'Unstable' ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] != 'Unhinged' ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] != 'Unglued' ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] != 'Unsanctioned' ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] != 'Unfinity' ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Journey into Nyx Hero's Path" ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Theros Hero's Path" ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Born of the Gods Hero's Path" ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Mystery Booster" ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Mystery Booster Playtest Cards 2019" ]
card_set_drop_tokens = card_set_drop_tokens[card_set_drop_tokens['set_name'] !=  "Mystery Booster Playtest Cards 2021" ]





In [12]:
card_set_drop_tokens.columns.values.tolist()

['object',
 'id',
 'oracle_id',
 'multiverse_ids',
 'mtgo_id',
 'mtgo_foil_id',
 'tcgplayer_id',
 'cardmarket_id',
 'name',
 'lang',
 'released_at',
 'uri',
 'scryfall_uri',
 'layout',
 'highres_image',
 'image_status',
 'image_uris',
 'mana_cost',
 'cmc',
 'type_line',
 'oracle_text',
 'power',
 'toughness',
 'colors',
 'color_identity',
 'keywords',
 'legalities',
 'games',
 'reserved',
 'foil',
 'nonfoil',
 'finishes',
 'oversized',
 'promo',
 'reprint',
 'variation',
 'set_id',
 'set',
 'set_name',
 'set_type',
 'set_uri',
 'set_search_uri',
 'scryfall_set_uri',
 'rulings_uri',
 'prints_search_uri',
 'collector_number',
 'digital',
 'rarity',
 'flavor_text',
 'card_back_id',
 'artist',
 'artist_ids',
 'illustration_id',
 'border_color',
 'frame',
 'full_art',
 'textless',
 'booster',
 'story_spotlight',
 'edhrec_rank',
 'penny_rank',
 'prices',
 'related_uris',
 'all_parts',
 'promo_types',
 'arena_id',
 'preview',
 'security_stamp',
 'produced_mana',
 'watermark',
 'frame_effects'

In [13]:
undesired_columns = [
    'object',
    'multiverse_ids',
    'mtgo_id',
    'mtgo_foil_id',
    'tcgplayer_id',
    'cardmarket_id',
    'lang',
    'released_at',
    'uri',
    'scryfall_uri',
    'layout',
    'highres_image',
    'image_status',
    'image_uris',
    'games',
    'foil',
    'nonfoil',
    'finishes',
    'oversized',
    'promo',
    'variation',
    'set_id',
    'set',
    'set_name',
    'set_type',
    'set_uri',
    'set_search_uri',
    'scryfall_set_uri',
    'rulings_uri',
    'prints_search_uri',
    'collector_number',
    'card_back_id',
    'artist',
    'artist_ids',
    'illustration_id',
    'border_color',
    'frame',
    'full_art',
    'textless',
    'booster',
    'story_spotlight',
    'prices',
    'related_uris',
    'all_parts',
    'promo_types',
    'arena_id',
    'preview',
    'security_stamp',
    'watermark',
    'frame_effects',
    'card_faces',
    'tcgplayer_etched_id',
    'attraction_lights',
    'variation_of',
    'printed_type_line',
    'printed_text',
    'printed_name',
    'content_warning',
    'flavor_name',
    'reprint',
    'digital',
    'color_indicator',
    'edhrec_rank',
    'penny_rank',
    'legalities',
    'reserved',
    'rarity',
    'life_modifier',
    'hand_modifier',
    'color_identity'
]

In [14]:
card_set_cleaned_undesired_columns = card_set_drop_tokens.drop(columns=undesired_columns)
print(f'Total de colunas original ---> {card_set_drop_tokens.shape[1]} colunas')
print(f'Total de colunas após limpeza ---> {card_set_cleaned_undesired_columns.shape[1]} colunas')
print(f'Número de colunas removidas ---> {len(undesired_columns)} colunas')


Total de colunas original ---> 84 colunas
Total de colunas após limpeza ---> 14 colunas
Número de colunas removidas ---> 70 colunas


In [15]:
card_set_cleaned_undesired_columns.columns.values.tolist()

['id',
 'oracle_id',
 'name',
 'mana_cost',
 'cmc',
 'type_line',
 'oracle_text',
 'power',
 'toughness',
 'colors',
 'keywords',
 'flavor_text',
 'produced_mana',
 'loyalty']

In [16]:
card_set_without_duplicates = card_set_cleaned_undesired_columns.drop_duplicates(subset='oracle_id')
print(f'Total de cartas ANTES da eliminação das duplicatas ---> {len(card_set_cleaned_undesired_columns)}')
print(f'Total de cartas APÓS da eliminação das duplicatas ---> {len(card_set_without_duplicates)}')
print(f'número de duplicatas removidas ---> {len(card_set_cleaned_undesired_columns)-len(card_set_without_duplicates)}')

Total de cartas ANTES da eliminação das duplicatas ---> 31271
Total de cartas APÓS da eliminação das duplicatas ---> 26866
número de duplicatas removidas ---> 4405


In [17]:
mtg_card_types = [
    'Artifact',
    'Battle ',
    'Creature',
    'Enchantment',
    'Land',
    'Planeswalker',
    'Instant',
    'Sorcery'
]

card_set_standardized_types = card_set_without_duplicates[card_set_without_duplicates['type_line'].str.contains('|'.join(mtg_card_types))]
print(f'Total de cartas ANTES da eliminação de tipos não desejados ---> {len(card_set_without_duplicates)}')
print(f'Total de cartas APÓS da eliminação de tipos não desejados ---> {len(card_set_standardized_types)}')
print(f'Número de cartas com tipos não desejados removidas ---> {len(card_set_without_duplicates)-len(card_set_standardized_types)}')

Total de cartas ANTES da eliminação de tipos não desejados ---> 26866
Total de cartas APÓS da eliminação de tipos não desejados ---> 25194
Número de cartas com tipos não desejados removidas ---> 1672


In [18]:
double_faced_cards_dataset = card_set_standardized_types[card_set_standardized_types['type_line'].str.contains('//')]
double_faced_cards_dataset.shape

(539, 14)

In [19]:
def getDoublefacedCardData(card_id):
    card = next(x for x in cards_json if x['id'] == card_id)

    front_face = card['card_faces'][0]
    back_face = card['card_faces'][1]
    
    return (
        {
            'id': card['id'],
            'oracle_id': card['oracle_id'],
            'mana_cost': front_face['mana_cost'] if 'mana_cost' in front_face else None,
            'cmc': card['cmc'],
            'type_line': front_face['type_line'],
            'oracle_text': front_face['oracle_text'] if 'oracle_text' in front_face else None,
            'power': front_face['power'] if 'power' in front_face else None,
            'toughness': front_face['toughness'] if 'toughness' in front_face else None,
            'colors': front_face['colors'] if 'colors' in front_face else None,
            'color_identity': card['color_identity'],
            'keywords': card['keywords'],
            'legalities': card['legalities'],
            'reserved': card['reserved'],
            'rarity': card['rarity'],
            'flavor_text': front_face['flavor_text'] if 'flavor_text' in front_face else None,
            'produced_mana': front_face['loyalty'] if 'loyalty' in front_face else None,
            'loyalty': front_face['loyalty'] if 'loyalty' in front_face else None,
            'life_modifier': front_face['life_modifier'] if 'life_modifier' in front_face else None,
            'hand_modifier': front_face['hand_modifier'] if 'hand_modifier' in front_face else None
        },
        {
            'id': card['id'],
            'oracle_id': card['oracle_id'],
            'mana_cost': back_face['mana_cost'] if 'mana_cost' in back_face else None,
            'cmc': card['cmc'],
            'type_line': back_face['type_line'],
            'oracle_text': back_face['oracle_text'] if 'oracle_text' in back_face else None,
            'power': back_face['power'] if 'power' in back_face else None,
            'toughness': back_face['toughness'] if 'toughness' in back_face else None,
            'colors': back_face['colors'] if 'colors' in back_face else None,
            'color_identity': card['color_identity'],
            'keywords': card['keywords'],
            'legalities': card['legalities'],
            'reserved': card['reserved'],
            'rarity': card['rarity'],
            'flavor_text': back_face['flavor_text'] if 'flavor_text' in back_face else None,
            'produced_mana': back_face['loyalty'] if 'loyalty' in back_face else None,
            'loyalty': back_face['loyalty'] if 'loyalty' in back_face else None,
            'life_modifier': back_face['life_modifier'] if 'life_modifier' in back_face else None,
            'hand_modifier': back_face['hand_modifier'] if 'hand_modifier' in back_face else None
        }
    )


In [20]:
double_faced_cards_id_list = double_faced_cards_dataset.id.tolist()
print(f'Total de cartas com face dupla ---> { len(double_faced_cards_id_list)} cartas')

Total de cartas com face dupla ---> 539 cartas


In [21]:
double_faced_cards_splited = []

for id in double_faced_cards_id_list:
    double_faced_cards_splited.extend(getDoublefacedCardData(id))

print(f'Montante de cartas a ser adicionado ao dataframe ---> {len(double_faced_cards_splited)} cartas')

Montante de cartas a ser adicionado ao dataframe ---> 1078 cartas


In [22]:
# print(f'{dropped_double_faced_cards_dataset}')

cards_to_drop = card_set_standardized_types.loc[card_set_standardized_types['id'].isin(double_faced_cards_id_list)].index
total_cards_to_drop = len(cards_to_drop)
print(f'Número de registros a serem removidos ---> {total_cards_to_drop} cartas')

dropped_double_faced_cards_dataset = card_set_standardized_types.drop(cards_to_drop)

total_cards_after_drop  = len(dropped_double_faced_cards_dataset.index)
total_cards_before_drop = len(card_set_standardized_types.index)

print(f'Total de cartas ANTES da remoção das cartas com face dupla ---> {total_cards_before_drop} cartas')
print(f'Total de cartas APÓS da remoção das cartas com face dupla ---> {total_cards_after_drop} cartas')
print(f'Total de cartas cartas face dupla removidas ---> { total_cards_before_drop - total_cards_after_drop} cartas')

dropped_double_faced_cards_dataset._append(double_faced_cards)
dropped_double_faced_cards_dataset.shape

Número de registros a serem removidos ---> 539 cartas
Total de cartas ANTES da remoção das cartas com face dupla ---> 25194 cartas
Total de cartas APÓS da remoção das cartas com face dupla ---> 24655 cartas
Total de cartas cartas face dupla removidas ---> 539 cartas


(24655, 14)

In [23]:
final_cards_dataset = dropped_double_faced_cards_dataset.copy()
final_cards_dataset.shape

(24655, 14)

### Carga e tratamento do conjunto de dados de regras expandidas

In [24]:
rules = pd.read_json('data/rulings-20230612090033.json')
rules

,object,oracle_id,source,published_at,comment
0,ruling,0004ebd0-dfd6-4276-b4a6-de0003e94237,wotc,2004-10-04,"If there are two of these on the battlefield, ..."
1,ruling,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,wotc,2019-08-23,The “commander tax” increases based on how man...
2,ruling,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,wotc,2019-08-23,Certain cards in other sets with the partner k...
3,ruling,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,wotc,2019-08-23,If your commander isn’t in the command zone (o...
4,ruling,000e5d65-96c3-498b-bd01-72b1a1991850,wotc,2004-10-04,The target loses just one of the listed abilit...
...,...,...,...,...,...
52331,ruling,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,wotc,2017-07-14,You divide the damage as you activate Samut’s ...
52332,ruling,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,wotc,2013-07-01,"Abilities that Slivers grant, as well as power..."
52333,ruling,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,wotc,2013-07-01,If the creature type of a Sliver changes so it...
52334,ruling,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,wotc,2021-03-19,Because damage remains marked on a creature un...


In [25]:
cleaned_rules = rules.drop(columns=['object', 'source', 'published_at'])
cleaned_rules

,oracle_id,comment
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,"If there are two of these on the battlefield, ..."
1,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,The “commander tax” increases based on how man...
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Certain cards in other sets with the partner k...
3,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,If your commander isn’t in the command zone (o...
4,000e5d65-96c3-498b-bd01-72b1a1991850,The target loses just one of the listed abilit...
...,...,...
52331,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,You divide the damage as you activate Samut’s ...
52332,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,"Abilities that Slivers grant, as well as power..."
52333,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,If the creature type of a Sliver changes so it...
52334,ffff90c3-63c4-4dee-a21d-6b2b113f4f80,Because damage remains marked on a creature un...


In [26]:
grouped_rules = cleaned_rules.groupby(['oracle_id']).agg(lambda x: x.tolist())
grouped_rules.shape

(14861, 1)

In [27]:
final_rules_dataset = grouped_rules.copy()

## Mesclagem dos conjuntos de dados

In [28]:
merged_dataset = final_cards_dataset.merge(final_rules_dataset, on='oracle_id', how='left')
merged_dataset.shape

(24655, 15)

In [29]:
final_merged_dataset = merged_dataset.drop(columns=['id', 'oracle_id'])
final_merged_dataset.shape

(24655, 13)

## Pré-processamento do dataset

### Correção do tipo

In [30]:
# final_merged_dataset[['type', 'subtype']] = final_merged_dataset['type_line'].str.split(' — ', expand=True)
# final_merged_dataset
splitted_type_dataset = final_merged_dataset.copy()
splitted_type_dataset[['type', 'subtype']] = splitted_type_dataset['type_line'].str.split('—', expand=True)
splitted_type_dataset.drop(columns=[ 'type_line'], inplace=True)
splitted_type_dataset.shape

(24655, 14)

### Cálculo de devoção

In [31]:
def calcDevotion(devotion):

    if not devotion:
        return 0

    devotion = str(devotion)

    devotions = []
    devotions.append(len(re.findall("[w]", devotion, re.IGNORECASE)))
    devotions.append(len(re.findall("[r]", devotion, re.IGNORECASE)))
    devotions.append(len(re.findall("[u]", devotion, re.IGNORECASE)))
    devotions.append(len(re.findall("[g]", devotion, re.IGNORECASE)))
    devotions.append(len(re.findall("[b]", devotion, re.IGNORECASE)))

    return max(devotions)

In [32]:
devotion_data_set = splitted_type_dataset.copy()
devotion_data_set['devotion'] = devotion_data_set.apply(lambda x : calcDevotion(x.mana_cost), axis=1)
devotion_data_set.shape

(24655, 15)

### Cálculo de mana phyrexiano

In [33]:
def checkPhyrexianMana(manaCost):

    if not manaCost:
        return 0

    return 1 if re.search('P', manaCost) else 0


In [34]:
phyrexian_mana_data_set = devotion_data_set.copy()
phyrexian_mana_data_set['phyrexian_mana'] = phyrexian_mana_data_set.apply(lambda x : checkPhyrexianMana(x.mana_cost), axis=1)
phyrexian_mana_data_set.drop(columns='mana_cost', inplace=True)
phyrexian_mana_data_set.shape

(24655, 15)

### Cálculo do _flavor text_

In [35]:
def calcTextLen(txt):
    
    if isinstance(txt, str):
        return len(txt)

    return 0

In [36]:
flavor_data_set = phyrexian_mana_data_set.copy()
flavor_data_set['flavor_size'] = flavor_data_set.apply(lambda x : calcTextLen(x.flavor_text), axis=1)
flavor_data_set.drop(columns=['flavor_text'], inplace=True)
flavor_data_set.shape

(24655, 15)

### Cálculo do número de habilidades

In [37]:
def calcAbilities(key_words: list):

    if key_words :
        return len(key_words)
        
    return 0

In [38]:
abilities_dataset = flavor_data_set.copy()
abilities_dataset['abilities'] = abilities_dataset.apply(lambda row: calcAbilities(row['keywords']), axis=1)
abilities_dataset.drop(columns=['keywords'], inplace=True)
abilities_dataset.shape

(24655, 15)

### Cálculo de produção de mana

Aqui haviam duas abordagens: 

1. Assumir um booleano que representa se uma carta é ou não capaz de produzir mana;
2. Definir quantas cores diferentes de mana uma dada carta pode gerar (sendo, portanto, um valor numérico - e não lógico como acima - que pode assumir o intervalo entre 0 e 5, inclusos)

Num primeiro momento, optou-se pela abordagem 1.

In [39]:
def canProduceMana(producedMana):

    if isinstance(producedMana, list):
        return 1 if len(producedMana) > 0 else 0

    return 0 if not producedMana or math.isnan(producedMana) else 0


In [40]:
produced_mana_dataset = abilities_dataset.copy()
produced_mana_dataset['produced_mana'] = produced_mana_dataset.apply(lambda row: canProduceMana(row['produced_mana']), axis=1)

In [41]:
produced_mana_dataset.columns

Index(['name', 'cmc', 'oracle_text', 'power', 'toughness', 'colors',
       'produced_mana', 'loyalty', 'comment', 'type', 'subtype', 'devotion',
       'phyrexian_mana', 'flavor_size', 'abilities'],
      dtype='object')

### Substituição do nome da carta

In [42]:
def replaceCardName(card_name, oracle_text) :

    if oracle_text and pd.notna(oracle_text) and card_name and pd.notna(card_name):
        return oracle_text.replace(card_name, 'CARDNAME')

In [43]:
card_replaced_name_by_token_dataset = produced_mana_dataset.copy()
card_replaced_name_by_token_dataset['oracle_text'] = card_replaced_name_by_token_dataset.apply(lambda row: replaceCardName(row['name'], row['oracle_text']), axis=1)
card_replaced_name_by_token_dataset.drop(columns='name', inplace=True)
card_replaced_name_by_token_dataset.shape

(24655, 14)

### Concatenação do texto de regras

In [44]:
def joinCardRules(cardRules):

    if isinstance(cardRules, list):
        return ' '.join(cardRules)

In [45]:
joined_rules_dataset = card_replaced_name_by_token_dataset.copy()
joined_rules_dataset['comment'] = joined_rules_dataset.apply(lambda row: joinCardRules(row['comment']), axis=1)

### Tratamento de Tipos e Supertipos

In [46]:
def getSuperTypes(type) :

    if type :
        supertype_type = type.strip().split(" ")

        mtg_card_types = [
            'Artifact',
            'Battle ',
            'Creature',
            'Enchantment',
            'Land',
            'Planeswalker',
            'Instant',
            'Sorcery'
        ]

        supertypes = supertype_type[:-1]
        types = supertype_type[-1:]

        supertypes = [x for x in supertypes if x not in mtg_card_types]
        
        return  (supertypes, types)

In [47]:
supertypes_and_types_dataset = joined_rules_dataset.copy()
supertypes_with_types_tuple = supertypes_and_types_dataset.apply(lambda row: getSuperTypes(row['type']), axis=1)
supertypes_and_types_dataset['supertype'] = supertypes_with_types_tuple.apply(lambda row: row[0])
supertypes_and_types_dataset['type'] = supertypes_with_types_tuple.apply(lambda row: row[1][0])
supertypes_and_types_dataset.tail()

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,comment,type,subtype,devotion,phyrexian_mana,flavor_size,abilities,supertype
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,[G],0,NaN,Herald of the Pantheon can reduce alternative ...,Creature,Centaur Shaman,1,0,100,0,[]
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",NaN,NaN,[],1,NaN,None,Land,None,0,0,86,0,[]
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,NaN,NaN,[U],0,NaN,"If you're at 1 life or less, you can't pay 2 l...",Sorcery,None,1,1,0,1,[]
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,[B],0,NaN,If there are no other creatures on the battlef...,Creature,Spirit,2,0,104,1,[]
24654,3.0,Flying,2,3,[W],0,NaN,None,Creature,Vampire Soldier,2,0,107,1,[]


### Recuperando as Dummies Variables

In [48]:
dummies_dataset = supertypes_and_types_dataset.copy()

mlb = MultiLabelBinarizer()

splited_colors = pd.DataFrame(mlb.fit_transform(dummies_dataset['colors']), columns=mlb.classes_, index=dummies_dataset.index)
splited_colors

,B,G,R,U,W
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,0,0,0,1
4,0,0,0,1,0
...,...,...,...,...,...
24650,0,1,0,0,0
24651,0,0,0,0,0
24652,0,0,0,1,0
24653,1,0,0,0,0


In [49]:
dummies_dataset_colors = dummies_dataset.join(splited_colors)
dummies_dataset_colors

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,comment,type,subtype,devotion,phyrexian_mana,flavor_size,abilities,supertype,B,G,R,U,W
0,6.0,All Sliver creatures have double strike.,3,3,[R],0,NaN,"Abilities that Slivers grant, as well as power...",Creature,Sliver,1,0,228,0,[],0,0,1,0,0
1,2.0,"When CARDNAME enters the battlefield, you may ...",2,2,[W],0,NaN,You may target Kor Outfitter with its ability....,Creature,Kor Soldier,2,0,71,0,[],0,0,0,0,1
2,3.0,"Flying\nWhen CARDNAME enters the battlefield, ...",1,2,[U],0,NaN,If a resolving spell or ability instructs a sp...,Creature,Siren Pirate,1,0,0,2,[],0,0,0,1,0
3,1.0,"When CARDNAME dies, put a +1/+1 counter on tar...",2,1,[W],0,NaN,If a Knight you control is dealt lethal damage...,Creature,Human Knight,1,0,71,0,[],0,0,0,0,1
4,3.0,"Whenever you draw your second card each turn, ...",3,1,[U],0,NaN,If a spell or ability causes you to put a card...,Creature,Fish,1,0,99,0,[],0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,[G],0,NaN,Herald of the Pantheon can reduce alternative ...,Creature,Centaur Shaman,1,0,100,0,[],0,1,0,0,0
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",NaN,NaN,[],1,NaN,None,Land,None,0,0,86,0,[],0,0,0,0,0
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,NaN,NaN,[U],0,NaN,"If you're at 1 life or less, you can't pay 2 l...",Sorcery,None,1,1,0,1,[],0,0,0,1,0
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,[B],0,NaN,If there are no other creatures on the battlef...,Creature,Spirit,2,0,104,1,[],1,0,0,0,0


In [50]:
splited_supertype = pd.DataFrame(mlb.fit_transform(dummies_dataset['supertype']), columns=mlb.classes_, index=dummies_dataset.index)
dummies_dataset_colors = dummies_dataset.join(splited_supertype)
dummies_dataset_colors

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,comment,type,subtype,devotion,phyrexian_mana,flavor_size,abilities,supertype,Basic,Legendary,Snow,Tribal,World
0,6.0,All Sliver creatures have double strike.,3,3,[R],0,NaN,"Abilities that Slivers grant, as well as power...",Creature,Sliver,1,0,228,0,[],0,0,0,0,0
1,2.0,"When CARDNAME enters the battlefield, you may ...",2,2,[W],0,NaN,You may target Kor Outfitter with its ability....,Creature,Kor Soldier,2,0,71,0,[],0,0,0,0,0
2,3.0,"Flying\nWhen CARDNAME enters the battlefield, ...",1,2,[U],0,NaN,If a resolving spell or ability instructs a sp...,Creature,Siren Pirate,1,0,0,2,[],0,0,0,0,0
3,1.0,"When CARDNAME dies, put a +1/+1 counter on tar...",2,1,[W],0,NaN,If a Knight you control is dealt lethal damage...,Creature,Human Knight,1,0,71,0,[],0,0,0,0,0
4,3.0,"Whenever you draw your second card each turn, ...",3,1,[U],0,NaN,If a spell or ability causes you to put a card...,Creature,Fish,1,0,99,0,[],0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,[G],0,NaN,Herald of the Pantheon can reduce alternative ...,Creature,Centaur Shaman,1,0,100,0,[],0,0,0,0,0
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",NaN,NaN,[],1,NaN,None,Land,None,0,0,86,0,[],0,0,0,0,0
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,NaN,NaN,[U],0,NaN,"If you're at 1 life or less, you can't pay 2 l...",Sorcery,None,1,1,0,1,[],0,0,0,0,0
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,[B],0,NaN,If there are no other creatures on the battlef...,Creature,Spirit,2,0,104,1,[],0,0,0,0,0


In [51]:
dummies_dataset = dummies_dataset_colors.drop(columns=['supertype'])
dummies_dataset['colors'] = dummies_dataset['colors'].apply(lambda x: len(x))

In [52]:
# pd.DataFrame(mlb.fit_transform(dummies_dataset['type']), columns=mlb.classes_, index=dummies_dataset.index)

# enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
columns = list(dummies_dataset['type'].unique())
enc = LabelBinarizer()
dummies_dataset_type = pd.DataFrame(enc.fit_transform(dummies_dataset['type'].values.reshape(-1,1)), columns=[columns[i] for i in range(len(columns))])

In [53]:
dummies_dataset = dummies_dataset.join(dummies_dataset_type)
dummies_dataset

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,comment,type,subtype,devotion,phyrexian_mana,flavor_size,abilities,Basic,Legendary,Snow,Tribal,World,Creature,Instant,Land,Artifact,Planeswalker,Sorcery,Enchantment,Equipment
0,6.0,All Sliver creatures have double strike.,3,3,1,0,NaN,"Abilities that Slivers grant, as well as power...",Creature,Sliver,1,0,228,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2.0,"When CARDNAME enters the battlefield, you may ...",2,2,1,0,NaN,You may target Kor Outfitter with its ability....,Creature,Kor Soldier,2,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3.0,"Flying\nWhen CARDNAME enters the battlefield, ...",1,2,1,0,NaN,If a resolving spell or ability instructs a sp...,Creature,Siren Pirate,1,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1.0,"When CARDNAME dies, put a +1/+1 counter on tar...",2,1,1,0,NaN,If a Knight you control is dealt lethal damage...,Creature,Human Knight,1,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3.0,"Whenever you draw your second card each turn, ...",3,1,1,0,NaN,If a spell or ability causes you to put a card...,Creature,Fish,1,0,99,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,1,0,NaN,Herald of the Pantheon can reduce alternative ...,Creature,Centaur Shaman,1,0,100,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",NaN,NaN,0,1,NaN,None,Land,None,0,0,86,0,0,0,0,0,0,0,0,0,0,0,1,0,0
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,NaN,NaN,1,0,NaN,"If you're at 1 life or less, you can't pay 2 l...",Sorcery,None,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,1,0,NaN,If there are no other creatures on the battlef...,Creature,Spirit,2,0,104,1,0,0,0,0,0,0,1,0,0,0,0,0,0


Optou-se pelo descarte de se fazer dummies a partir da coluna de subtipos dado ao número elevado de colunas que seriam gerados:

In [54]:
subtypes_columns = list(dummies_dataset['subtype'].unique())
len(subtypes_columns)

1816

In [55]:
filled_dataset = dummies_dataset.copy()
filled_dataset.replace(np.nan, 0, inplace=True)

In [56]:
filled_dataset['oracle_text'].replace(0, "", inplace=True)

In [57]:
filled_dataset['comment'].replace(0, "", inplace=True)

In [58]:
filled_dataset['oracle_text'] = filled_dataset['oracle_text'] + ' ' + filled_dataset['comment']
filled_dataset

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,comment,type,subtype,devotion,phyrexian_mana,flavor_size,abilities,Basic,Legendary,Snow,Tribal,World,Creature,Instant,Land,Artifact,Planeswalker,Sorcery,Enchantment,Equipment
0,6.0,All Sliver creatures have double strike. Abili...,3,3,1,0,0,"Abilities that Slivers grant, as well as power...",Creature,Sliver,1,0,228,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2.0,"When CARDNAME enters the battlefield, you may ...",2,2,1,0,0,You may target Kor Outfitter with its ability....,Creature,Kor Soldier,2,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3.0,"Flying\nWhen CARDNAME enters the battlefield, ...",1,2,1,0,0,If a resolving spell or ability instructs a sp...,Creature,Siren Pirate,1,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1.0,"When CARDNAME dies, put a +1/+1 counter on tar...",2,1,1,0,0,If a Knight you control is dealt lethal damage...,Creature,Human Knight,1,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3.0,"Whenever you draw your second card each turn, ...",3,1,1,0,0,If a spell or ability causes you to put a card...,Creature,Fish,1,0,99,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,1,0,0,Herald of the Pantheon can reduce alternative ...,Creature,Centaur Shaman,1,0,100,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",0,0,0,1,0,,Land,0,0,0,86,0,0,0,0,0,0,0,0,0,0,0,1,0,0
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,0,0,1,0,0,"If you're at 1 life or less, you can't pay 2 l...",Sorcery,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,1,0,0,If there are no other creatures on the battlef...,Creature,Spirit,2,0,104,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [59]:
filled_dataset.drop(columns=['comment', 'type', 'subtype'], inplace=True)
filled_dataset

,cmc,oracle_text,power,toughness,colors,produced_mana,loyalty,devotion,phyrexian_mana,flavor_size,abilities,Basic,Legendary,Snow,Tribal,World,Creature,Instant,Land,Artifact,Planeswalker,Sorcery,Enchantment,Equipment
0,6.0,All Sliver creatures have double strike. Abili...,3,3,1,0,0,1,0,228,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2.0,"When CARDNAME enters the battlefield, you may ...",2,2,1,0,0,2,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,3.0,"Flying\nWhen CARDNAME enters the battlefield, ...",1,2,1,0,0,1,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1.0,"When CARDNAME dies, put a +1/+1 counter on tar...",2,1,1,0,0,1,0,71,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3.0,"Whenever you draw your second card each turn, ...",3,1,1,0,0,1,0,99,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24650,2.0,Enchantment spells you cast cost {1} less to c...,2,2,1,0,0,1,0,100,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24651,0.0,"{T}: Add {C}.\n{1}, {T}: Add one mana of any c...",0,0,0,1,0,0,0,86,0,0,0,0,0,0,0,0,0,0,0,1,0,0
24652,4.0,({U/P} can be paid with either {U} or 2 life.)...,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
24653,5.0,"Morbid — When CARDNAME enters the battlefield,...",4,4,1,0,0,2,0,104,1,0,0,0,0,0,0,1,0,0,0,0,0,0


## Salvando o conjunto de dados

In [60]:
import pyarrow.feather as feather
import pyarrow as pa
import pyarrow.parquet as pq

In [70]:
str_dataset = filled_dataset.astype(str)

In [72]:
str_dataset.to_feather('data/processed_data')